<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/analyses/descriptive_analysis_v3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deskriptive Analyse
Eine deskriptive Analyse des gesamten annotierten Datensets. (Paragraphen), dh. alle Kategorien inkl. "Domestic Violence".
Mit Fokus auf:
1. regionale vs. nationale Zeitungen
2. Ressorts
3. Anzahl der Paragraphen im Zeitverlauf
4. Länge der Paragraphen
5. Annotationen

Allgemeine Daten:
- Alle Artikel: 60'866'588
- Alle Artikel (DE): 50'964'697
- Alle gefilterten Artikel 25'465
OLD:
<!-- - Alle annotierten Paragraphen: 3'497
- Alle annotierten Paragraphen zum Thema Häusliche Gewalt: 1'268
- Alle annotierten Paragraphen mit relevantem Label: 127 -->

### Import

In [91]:
# packages
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm
from ast import literal_eval
import plotly.graph_objects as go
import plotly.express as px
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [92]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
# change cwd
%cd drive/MyDrive/Work/Frontline/data
#%cd /content/drive/MyDrive/data/

[Errno 2] No such file or directory: 'drive/MyDrive/Work/Frontline/data'
/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [94]:
from scripts import annotations

### Data

In [95]:
# list of dfs with all annotated datasets
dfs={}
for doc in os.listdir("annotated/new_ontology"):
  if doc.endswith(".json"):
    #read json data
    json_data=json.load(open("annotated/new_ontology/"+doc, encoding="utf-8"))
    #convert to dataframe
    data=pd.DataFrame(json_data["documents"])
    data.loc[:,"file"]=doc
    dfs[doc]=data

dataV2=pd.concat(dfs,ignore_index=True)

Extract:artikel_id, titel, annotations


In [96]:
dataV2.loc[:,"artikel_id"]=dataV2.attributes_flat.apply(lambda x: x["artikel_id"])
dataV2.loc[:,"name"]=dataV2.attributes_flat.apply(lambda x: x["name"])
dataV2.loc[:,"titel"]=dataV2.attributes_flat.apply(lambda x: x["titel"])
dataV2.loc[:,"ressort"]=dataV2.attributes_flat.apply(lambda x: x["ressort"])
dataV2.loc[:,"annotations"]=dataV2.loc[:,"annotations"].apply(annotations.extract_annotations)


In [97]:
dataV1=pd.read_csv("annotated/230621_all_annotationsV1.csv", index_col=0,converters={"annotations":literal_eval})

In [98]:
data=pd.concat([dataV1,dataV2])

In [99]:
data=data.sort_values("file").drop_duplicates(["text","artikel_id"],keep="last")

In [100]:
data.loc[:,"annotations"]=data.apply(annotations.ground_truth_filter,min_coannotation=1,min_similarity=-1, similarity="dice",axis=1)


### General

In [101]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5628 entries, 0 to 4499
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   artikel_id       5628 non-null   object 
 1   text             5628 non-null   object 
 2   name             5628 non-null   object 
 3   datum            1225 non-null   float64
 4   ressort          5130 non-null   object 
 5   annotations      2564 non-null   object 
 6   attributes_flat  5608 non-null   object 
 7   file             5628 non-null   object 
 8   artikel_order    1104 non-null   float64
 9   jaccard          26 non-null     float64
 10  dice             26 non-null     float64
 11  id               4403 non-null   object 
 12  titel            4403 non-null   object 
dtypes: float64(4), object(9)
memory usage: 615.6+ KB


### OLD: Übersicht:

In [102]:
fig = go.Figure(go.Funnel(y=["Artikel nach Keyword-Filter","Artikel nach allen Filtern","Annotierte Artikel \n (jeweils 1 Paragraph)","Annotierte Paragraphen zu Häuslicher Gewalt","Annotierte Paragraphen der 3 schädlichen Kategorien" ],
                    x=[71256,25465, 5628, 2564,278]))
fig.show()

### Constants

In [103]:
regional_newspapers=['Aachener Nachrichten',
 'Aachener Zeitung',
 'Alb Bote',
 'Aller-Zeitung',
 'Allgemeine Zeitung Mainz-Rheinhessen',
 'Badische Zeitung',
 'Barmstedter Zeitung',
 'Bayerische Rundschau',
 'Bergedorfer Zeitung',
 'Bergische Morgenpost',
 'Berliner Kurier',
 'Berliner Morgenpost',
 'Berliner Zeitung',
 'Bersenbrücker Kreisblatt',
 'Bonner General-Anzeiger',
 'Bote vom Haßgau',
 'Bramscher Nachrichten',
 'Braunschweiger Zeitung',
 'Coburger Tageblatt',
 'DIE KITZINGER',
 'Darmstädter Echo',
 'Der Prignitzer',
 'Dresden am Wochenende',
 'Dresdner Morgenpost',
 'Dresdner Neueste Nachrichten',
 'Döbelner Allgemeine Zeitung',
 'Eichsfelder Tageblatt',
 'Ems-Zeitung',
 'Freie Presse',
 'Fränkischer Tag',
 'Gießener Anzeiger',
 'Göttinger Tageblatt',
 'Haller Tagblatt',
 'Hamburger Abendblatt',
 'Hamburger Morgenpost',
 'HarzKurier',
 'Heilbronner Stimme',
 'Hessische Niedersächsische Allgemeine',
 'Hildesheimer Allgemeine Zeitung',
 'Hochheimer Zeitung',
 'Hofheimer Zeitung',
 'Hohenloher Tagblatt',
 'Hohenzollerische Zeitung',
 'Husumer Nachrichten',
 'Höchster Kreisblatt',
 'Idsteiner Zeitung',
 'Kieler Nachrichten',
 'Kirner Zeitung',
 'Kreis-Anzeiger',
 'Lampertheimer Zeitung',
 'Landshuter Zeitung',
 'Lausitzer Rundschau - Elbe-Elster-Rundschau',
 'Lauterbacher Anzeiger',
 'Le Monde Diplomatique',
 'Leipziger Volkszeitung',
 'Lingener Tagespost',
 'Lübecker Nachrichten',
 'Magdeburger General-Anzeiger','Münsterland Zeitung',
 'Nahe Zeitung',
 'Nassauische Neue Presse',
 'Neue Osnabrücker Zeitung','Neue Ruhr/Rhein Zeitung',
 'Neue Westfälische',
 'Neue Württembergische Zeitung',
 'Neuss-Grevenbroicher Zeitung',
 'Neue Presse Hannover',
 'Magdeburger Volksstimme','Main-Post',
 'Main-Spitze','Nordbayerischer Kurier',
 'Norddeutsche Neueste Nachrichten',
 'Norddeutsche Rundschau','Nordkurier',
 'Nordwest-Zeitung',
 'Nürnberger Nachrichten','Oberhessische Zeitung',
 'Oeffentlicher Anzeiger',
 'Oschatzer Allgemeine Zeitung',
 'Osterländer Volkszeitung','Passauer Neue Presse',
 'Peiner Allgemeine Zeitung',
 'Reutlinger General-Anzeiger',
 'Rhein-Hunsrück-Zeitung',
 'Rhein-Lahn-Zeitung',
 'Rhein-Zeitung','Ruhr Nachrichten',
 'Rundschau für den Schwäbischen Wald',
 'Rüsselsheimer Echo',
 'Saale-Zeitung',
 'Saarbrücker Zeitung',
 'Salzgitter-Zeitung','Thüringische Landeszeitung',
 'Trierischer Volksfreund',
 'Usinger Anzeiger',
 'Volksblatt Würzburg',
 'Volkszeitung Schweinfurt'
 'Ostsee-Zeitung',
 'Ostthüringer Zeitung','Schweinfurter Tagblatt',
 'Schweriner Volkszeitung','Ostsee-Zeitung',
 'Volkszeitung Schweinfurt',
 'Westerwälder Zeitung',
 'Wiesbadener Kurier',
 'Wiesbadener Tagblatt',
 'Schwäbische Zeitung',
 'Segeberger Zeitung',
 'Solinger Morgenpost','Nürnberger Zeitung',
 'Straubinger Tagblatt',
 'Sächsische Zeitung',
 'Südkurier',
 'Taunus Zeitung','Wittlager Kreisblatt',
 'Wolfenbütteler Zeitung',
 'Wolfsburger Allgemeine Zeitung',
 'Wolfsburger Nachrichten',
 'Wormser Zeitung',
 'Thüringer Allgemeine',
 'Straubinger Tagblatt'
 'Nürnberger Zeitung',
 'Kölnische Rundschau']
national_newspapers=['Audio Video Foto Bild','Börsen-Zeitung','F.A.Z. Einspruch',
 'F.A.Z. Wirtschaftswissenschaft','FOCUS',
 'Frankfurter Neue Presse','Handelsblatt',
 'Handelsblatt Morning Briefing',"Jüdische Allgemeine","Süddeutsche Zeitung",
 'Frankfurter Rundschau',"Hannoversche Allgemeine Zeitung", 'ZEIT Campus',
 'ZEIT Studienführer',
 'ZEIT Wissen','Kölner Stadt-Anzeiger',
 'taz.die tageszeitung',"LiteraturSPIEGEL",
 'Christ und Welt','B.Z.',"BILD","BUSINESS INSIDER DEUTSCHLAND",'Spiegel Start',
 'Sport Bild', 'WELT KOMPAKT',
 'WELT ONLINE','Meller Kreisblatt',
 'Meppener Tagespost',
 'Metzinger Uracher Volksblatt',
 'Mittelbayerische Zeitung','Westfalen-Blatt',
 'Westfalenpost',
 'Mitteldeutsche Zeitung',"Wirtschaftszeitung",
 'WELT am SONNTAG',
 'Stern', "Märkische Allgemeine","SÜDWEST PRESSE",'Westdeutsche Allgemeine Zeitung',
 'Westdeutsche Zeitung','Le Monde Diplomatique',"Westfälische Rundschau","Rheinische Post",'Münchner Merkur',"BILD am Sonntag","Abendzeitung",'DIE ZEIT online','ZEIT Geschichte','DIE ZEIT', "DER SPIEGEL","DIE WELT","EXPRESS","taz","SPIEGEL Plus",'Bild der Frau','DER SPIEGEL online']



### 1. Regional vs. National

In [104]:
data=data[~data.annotations.isna()]

In [105]:
%%capture
data.loc[:,"category"]=["regional" if name in regional_newspapers else "national" for name in data.name ]

In [106]:
fig=px.pie(names=list(data["category"].value_counts().keys()), title='Artikel zu Häuslicher Gewalt nach Kategorie: national/ regional',values=data["category"].value_counts())
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [107]:
fig.write_image("category.png")

In [108]:
print("Anzahl der vorkommenden Zeitungen: ",data.name.nunique())
top=20
print(f"\nTop {top} der vorkommenden Zeitungen:\n")
round(data.name.value_counts(normalize=True)[:top].mul(100),2).astype(str)+" %"

Anzahl der vorkommenden Zeitungen:  141

Top 20 der vorkommenden Zeitungen:



WELT ONLINE                24.69 %
Ruhr Nachrichten            8.81 %
BILD                        5.97 %
Schwäbische Zeitung         4.29 %
Südkurier                   2.93 %
Neue Westfälische           2.65 %
DIE WELT                    2.26 %
B.Z.                        1.95 %
Westfalen-Blatt              1.6 %
Nordwest-Zeitung            1.37 %
Hamburger Abendblatt        1.33 %
WELT am SONNTAG             1.25 %
Neue Ruhr/Rhein Zeitung     1.01 %
Bergedorfer Zeitung          0.9 %
Berliner Morgenpost         0.86 %
Münsterland Zeitung         0.86 %
Sächsische Zeitung          0.86 %
Der Prignitzer              0.82 %
Schweriner Volkszeitung     0.82 %
Bonner General-Anzeiger     0.82 %
Name: name, dtype: object

In [109]:
fig=px.bar(data.name.value_counts(normalize=True)[:20]*100,title="Zeitungen der annotierten Artikel",)
fig.update_layout(xaxis_title='Zeitung', yaxis_title='Anteil in %',paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")

In [110]:
fig.write_image("journals.png")

### 2. Ressorts

In [111]:
data.ressort=data.ressort.str.title()
ressort_rename={"PANORAMA":"Panorama","Titel":"Titelseite","Meinung Und Debatte":"Debatte","Aus Der Region":"Region",
                "Ausland":"Aus Aller Welt","Blick In Die Welt":"Aus Aller Welt","Antworten Und Debatte":"Debatte",
                "/Wn/Regiodesk Wb/Aus Aller Welt":"Aus Aller Welt","Rund Um Die Welt":"Aus Aller Welt", "Welt":"Aus Aller Welt","Weltspiegel":"Aus Aller Welt",
                "Welt Im Spiegel":"Aus Aller Welt",
                "Welt Aktuell":"Aus Aller Welt",
                "Deutschland":"Inland",
                "Land":"Inland",
                "F.A.Z. Einspruch":"Justiz",
                "Nrw Und Deutschland":"Inland"
}
data.ressort=data.ressort.replace(ressort_rename)


In [112]:
ressort_rename_regex={"[A-z +\&/\n,0-9;]*Lokal\S*[&äA-z \/]*":"Lokales",
                      "Ressort:\s":"",
                      "\S*[A-z \/\n]*Politik\S*[A-z \/\n]*":"Politik",
                      "\S*[A-z \&/\n]*Kultur\S*[A-z \/]*":"Kultur",
                      "Titel\S*[A-z \/]*":"Titelseite","\S*[A-z \&/\n]*Justiz\S*[&äA-z \/]*":"Justiz",
                      "\S*[A-z \&/\n]*Hintergrund\S*[&äA-z \/]*":"Hintergrund",
                      "\S*[A-z \&/\n]*Leser\S*[&äA-z \/]*":"Leser",
                      "\S*[A-z \&/\n]*Medien\S*[&äA-z \/]*":"Medien",
                      "\S*[A-z \&/\n]*Forum\S*[&äA-z \/]*":"Forum",
                      "[A-z +\&/\n,0-9\-;]*Meinung\S*[&äA-z \/]*":"Meinung",
                      "[A-z +\&/\n,0-9()Üß\-;]*Sport\S*[&äA-z \/]*":"Sport",
                      "[A-z ]*Aktuell\S*[&äA-z \/]*":"Aktuell",
                      "[A-z &ü\-]*Region\S*[\&\-äA-z \/]*":"Region",
                      "[A-z &ü\-]*region\S*[\&\-äA-z \/]*":"Region",
                      "[A-z &ü\-]*Panorama\S*[\&\-äA-z \/]*":"Panorama",
                      "(Hamburg|Dortmunder Zeitung|Lünen|Castrop-Rauxel|Werne|Berlin|Herford|Gütersloh|Bielefeld|Lübbecke|Konstanz|Ulm Und Neu-Ulm|Paderborn|Höxter|Selm|Thüringen|Die Stadtteil-Nachrichten|Blick Nach Dortmund|Hannover|Aus Den Kreisen|Regensbur|Bad Oeynhausen|Schwerter Zeitung)":"Lokales",
                      "(Mecklenburg-Vorpommern|Hessen|Franken|Wir Im Süden|Rheinland-Pfalz|Sachsen|Nrw|Region Und Mitteldeutschland|Niedersachsen & Der Norden|Schleswig-Holstein|Baden-Württemberg|Niedersachsen|Brandenburg)":"Region",
                      "(Ahaus|Schloss Holte|Dresden|Jever|Sonderveröffentlichung Dortmund|München|Waldshut-Tiengen|Oldenburg|Vreden|Singen|Büde|Bonn|Friesoythe)":"Lokales",
                      "(Olfen Vinnum|Wildeshausen|Cham|Bünde|Hegau|Moosburger Zeitung|Schleswig-Lokales|Kreis Steinburg|Kreis Göppingen|Hochrhein|Hildesheim)":"Lokales",






}
data.ressort=data.ressort.replace(ressort_rename_regex,regex=True)

In [113]:
data[data.annotations!={"Domestic Violence"}].shape

(278, 14)

In [114]:
fig=px.bar(data.ressort.replace("","NO RESSORT").value_counts()[:10],title="Ressorts der annotierten Artikel")
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [115]:
fig.write_image("ressorts.png")

In [116]:
print("Anzahl der vorkommenden Ressorts: ",data.ressort.nunique())

Anzahl der vorkommenden Ressorts:  221


In [117]:
# grouping <0.5% ressorts into one
top20_ressort=list(data.ressort.value_counts(normalize=True)[:15].keys())
mask=[res not in top20_ressort for res in data.ressort]
temp_data=data
temp_data.loc[mask,"ressort"]="Alle anderen Ressorts \n(<0.5%)"

In [118]:
fig = px.pie(names=temp_data.ressort.value_counts()[:15].keys(), title='Artikel zu Häuslicher Gewalt nach Ressort\n',values=list(temp_data.ressort.value_counts()[:15]))
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

### 3. Anzahl der Artikel im Zeitverlauf

In [119]:
data.datum=data.datum.apply(pd.to_datetime,format="%Y%m%d")

In [120]:
data_grouped=data.groupby(["datum"]).count()["artikel_id"]
data_grouped=pd.DataFrame(data_grouped).reset_index().rename(columns={"artikel_id":"Veröffentlichte Artikel (pro Tag)", "datum":"Datum"})

In [121]:
fig = px.line(data_grouped, y= "Veröffentlichte Artikel (pro Tag)",x="Datum", title='Veröffentlichte Artikel zum Thema Häusliche Gewalt')
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [124]:
data_grouped_monthly=data_grouped
data_grouped_monthly.Datum=[i +pd.offsets.MonthBegin(-1) for i in data_grouped_monthly.Datum]
data_grouped_monthly=data_grouped_monthly.groupby("Datum").sum().reset_index().rename(columns={"Veröffentlichte Artikel (pro Tag)":"Veröffentlichte Artikel (pro Monat)"})
fig = px.bar(data_grouped_monthly, y= "Veröffentlichte Artikel (pro Monat)",x="Datum", title='Veröffentlichte Artikel zum Thema Häusliche Gewalt',)
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [125]:
#fig.write_image("timeline.png")

In [126]:
data_grouped_res=data
data_grouped_res.datum=[i +pd.offsets.MonthBegin(-1) for i in data_grouped_res.datum]
data_grouped_res=data_grouped_res.groupby(["datum","ressort"]).count()[["artikel_id"]]
data_grouped_res=pd.DataFrame(data_grouped_res).reset_index().rename(columns={"artikel_id":"Veröffentlichte Artikel (pro Monat)", "datum":"Datum","ressort":"Ressort"})
fig = px.bar(data_grouped_res, y= "Veröffentlichte Artikel (pro Monat)",x="Datum", title='Veröffentlichte Artikel zum Thema Häusliche Gewalt',color="Ressort")
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [127]:
#fig.write_image("timeline_resorts.png")

In [128]:
data_grouped_cat=data
data_grouped_cat.datum=[i +pd.offsets.MonthBegin(-1) for i in data_grouped_cat.datum]
data_grouped_cat=data_grouped_cat.groupby(["datum","category"]).count()[["artikel_id"]]
data_grouped_cat=pd.DataFrame(data_grouped_cat).reset_index().rename(columns={"artikel_id":"Veröffentlichte Artikel (pro Monat)", "datum":"Datum","category":"Kategorie"})
fig = px.bar(data_grouped_cat, y= "Veröffentlichte Artikel (pro Monat)",x="Datum", title='Veröffentlichte Artikel zum Thema Häusliche Gewalt',color="Kategorie")
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [130]:
#fig.write_image("timeline_cat.png")

### 4. Länge von Paragraphen
-> beachte: Paragrphen mit Überlänge (>120 Wörter) wurden rausgefiltert

In [131]:
data["lengths"]=[len(i.split()) for i in data.text]

In [132]:
fig=px.histogram(data.rename(columns={"category":"Kategorie"}),"lengths", nbins=50, title="Länge von Paragraphen zum Thema häusliche Gewalt", color="Kategorie")
fig.update_layout(yaxis_title="Anzahl der Artikel")
fig.update_layout(xaxis_title="Anzahl der Wörter eines Paragraphen")
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [133]:
#fig.write_image("length_resort.png")

In [134]:
fig=px.bar(data.rename(columns={"lengths":"Länge"}).groupby("ressort")["Länge"].mean(),title="Länge der Paragraphen nach Ressort")
fig.update_layout(yaxis_title="Länge der Paragraphen (Wörter)")
fig.update_layout(xaxis_title="Ressorts")
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

## 5. Annotationen

#### Alle Annotationen

In [132]:
names=[list(i) for i in data.annotations.value_counts().keys()]

In [133]:
fig = px.pie(names=[["Skipped Annotations"]]+names, title='Alle Annotierten Paragraphen',values=[2229]+list(data.annotations.value_counts(normalize=False)))
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

#### Annotationen zum Thema Häusliche Gewalt

In [136]:
fig = px.pie(names=names, title='Labels der Annotierten Paragraphen',values=list(data.annotations.value_counts(normalize=False)))
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [139]:
fig = px.pie(names=names[1:], title='Labels der Annotierten Paragraphen',values=list(data.annotations.value_counts(normalize=False))[1:])
fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)")
fig.show()

In [140]:
fig.write_image("cats_dv.png")